# Maximum Likelihood for Data Distributions - Week 8  

In [1]:
import scipy.stats as st
import statsmodels.datasets
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
#https://www.statsmodels.org/devel/examples/index.html
dataf = statsmodels.datasets.cancer.load_pandas().data

In [ ]:
print(statsmodels.datasets.cancer.NOTE)

In [ ]:
dataf.head()

In [ ]:
cancer = dataf.cancer

In [ ]:
plt.plot(sorted(cancer)[::-1]) #sort number of cases in ascending or and then flip the graph around the vertical
plt.xlabel("Observation")
plt.ylabel("Cancers")

In [ ]:
plt.hist(dataf['cancer'], bins = 20);
plt.xlabel("Cancer Cases")
plt.ylabel("Frequency of Observations")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
ax1.plot(sorted(cancer)[::-1])
ax1.set_xlabel('observations')
ax1.set_ylabel('Cancers')

ax2.hist(cancer, bins=20)
ax2.set_xlabel('Cancer Cases')
ax2.set_ylabel('Frequency of Observations')

## This looks like an exponential decay so let's see if we can fit this to the data.  An exponential function's decay is goverened by the rate parameter λ.

## The Maximum Likelihood algorithms tries to estimate the value of λ  that maximizes the probabilty of observing data in an exponential data distribution. Effectively this can be shown to be the inverse of the mean of the data in the exponential distribution.

In [ ]:
# The mean for the number of cancer observations is According to this model, 
#S (number of days of survival) is an exponential random variable with the parameter λ, 
#and the observations si are sampled from this distribution. Let the sample mean be:

#C_mean = 1/n * ∑ci

C_mean = cancer.mean()
lambda_param = 1. / C_mean

## Let's now compare this exponential distribution to the actual data. But first we will have to use the numpy method linspace to scale the exponential function we are trying to fit to the scale of the actual data. Linspace takes a start point (0), a stop point (C_max), and the number of separations along the axis(301) 

In [ ]:
C_max = cancer.max() # maximum number of cancer cases in an histogram bin.
cancer_cases = np.linspace(0., C_max, 301)

## Let's now find the probability density function of this distribution by passing the pdf method the observations we set up above and then passing it the scale of the inverse of the lambda parameter.

In [ ]:
expo_dist = st.expon.pdf(cancer_cases, scale=1. / lambda_param)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4)) # doing this allows us to plot the graphs on top of each other.
ax.hist(cancer, bins=30) # plot the histogram and then in the next line plot the exponential dist over the top
ax.plot(cancer_cases, expo_dist * len(cancer) * C_max / 30,'-y', lw=5) # -y= yellow, lw = line width
ax.set_xlabel("Cancer Cases")
ax.set_ylabel("Frequency of Observations")

## This looks like quite a good fit but we really need a measure of how good our maximum likelihood estimation was. In order to do that we can get some measurement parameters by using a different SciPy method that also calculates the best exponential fit.

In [ ]:
expo_dist = st.expon
arguments = expo_dist.fit(cancer)
arguments


## The above 'fit' method, when passed the cancer cases column from the data frame, will return some arguments we which can then pass to the Kolmogorov-Smirnov test.This will test how well our chosen distribution type fits the actual data.

In [ ]:
st.kstest(cancer, expo_dist.cdf, arguments)

## When we use the KStest we usually set up a null hypothesis that states that the fitted distribution is a 'good-fit' but here we see tha the p-value is below 0.05 and therefore we have to reject the hypothesis for a 95% confidence interval. The exponential distribution is not a good fit for our data even though it looks as if it is.

## We can try another type of distribution that is used in calculating the lifetime of components before failure. It is called the Birnbaum-Sanders distribution.

In [ ]:
fLife_dist= st.fatiguelife
arg = fLife_dist.fit(cancer)
st.kstest(cancer, fLife_dist.cdf, arg)

## The p-value here is above 0.05 so we can accept the null hypothesis that this distribution is a good fit. We can plot it below and compare to the exponential distribution as well,

In [ ]:
fatLife_dist = fLife_dist.pdf(cancer_cases, *arg)

fig, ax = plt.subplots(1, 1, figsize=(12, 7))
ax.hist(cancer, bins = 50)
ax.plot(cancer_cases, fatLife_dist * len(cancer) * C_max / 50, '-g', lw=5, label='FLife')
ax.set_xlabel("Cancer Cases")
ax.set_ylabel("Observations")

## We can try and fit a normal distribution to this as well just to see what happens.

In [ ]:
dist_norm = st.norm.pdf(cancer_cases, scale=1. / lambda_param)


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4))
ax.hist(cancer, bins=30)
ax.plot(cancer_cases, dist_norm * len(cancer) * C_max / 30,
        '-r', lw=3)
ax.set_xlabel("Cancer Cases")
ax.set_ylabel("Observations")

In [ ]:
dist = st.norm
args = dist.fit(cancer)
args

In [ ]:
st.kstest(cancer, dist.cdf, args)

## Now let's try a set of numbers that might be more like a normal distribution.

In [ ]:
import pandas as pd

lst = [1, 4, 5, 7, 8, 8, 9, 11, 5, 2, 1, 3, 5, 4, 8, 9]


# Calling DataFrame constructor on list 
df = pd.DataFrame(lst, columns=['numbers']) 
df.head() 

In [ ]:
numbers = df.numbers

In [ ]:
plt.hist(df['numbers'], bins = 20);

In [ ]:
dist = st.norm
args = dist.fit(numbers)
args

In [ ]:
st.kstest(numbers, dist.cdf, args)